<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3><center>تمرین چهارم درس پردازش زبان‌های طبیعی</center></h3>
<h4><center>چالش تحلیل احساسات</center></h4>
<table width='100%' style="border: none;">
    <tr style="border: none; text-align: center;">
        <td style="border: none;"><h5>علیرضا بلال</h5></td>
        <td style="border: none;"><h5>زهرا رجالی</h5></td>
        <td style="border: none;"><h5>جواد راضی</h5></td>
    </tr>
        <tr style="border: none; text-align: center;">
        <td style="border: none;"><h5>400200881</h5></td>
        <td style="border: none;"><h5>401204716</h5></td>
        <td style="border: none;"><h5>401204354</h5></td>
    </tr>
</table>
<h5 style="font-size: 16px;"><center> بهار ۱۴۰۲ </center></h5>
<br/>
<hr/>
<br/>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar', 'B Lotus', 'Calibri'" size=3><div dir='rtl' align='justify'>
<b>
    فایل ژوپیتر این تمرین در کولب توسعه داده و تست شده‌است. این فایل هم در محیط کولب، هم با ایمیج داکر jupyter/datascience-notebook تست شده‌است و همه قطعه‌کدها خروجی مورد انتظار را می‌دهند. اگه در بازتولید خروجی بعضی سل‌ها، یا کدهای تمرین مشکلی وجود داشت، ممنون می‌شویم در صورت امکان به ما اطلاع دهید تا فایل را در محیطی که قابل اجرا است، اجرا نموده و خروجی را نمایش دهیم.
</b>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **خلاصه‌ای از نحوه انجام تمرین**
در این پروژه یک مدل تحلیل احساسات مبتنی بر جنبه را برای نظرات کاربران (به زبان فارسی) در یک وب‌سایت فیلم،  پیاده سازی می کنیم. مدل نهایی، متن نظر (نقد) کاربر،  و فهرستی از جنبه‌ها را به عنوان ورودی دریافت می‌کند و برای هر جنبه، نظر را از این جنبه سنجیده و احساس (Sentiment) را برای این جنبه طبقه‌بندی می‌کند.


به طور کلی، مراحل انجام خواسته‌های این تمرین به صورت زیر است:‌

1. پیش‌پردازش و نرمال‌سازی داده‌ها
2. تعریف و ترین‌کردن مدل
3. ارزیابی مدل
4. استخراج جملات مرتبط با هر جنبه، از کل متن نقد
5. پیاده‌سازی تابع نهایی طبقه‌بندی احساسات
6. ارزیابی تابع نهایی طبقه‌بندی احساسات

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## واردکردن و نصب کتاب‌خانه‌های مورد استفاده


In [1]:
%pip install transformers[torch]

try:
    import transformers
except:
    %pip install transformers

try:
    import ipywidgets
except:
    %pip install ipywidgets

try:
    import pandas as pd
except:
    %pip install pandas

try:
    import datasets
except:
    %pip install datasets

try:
    import matplotlib as mpl
except:
    %pip install matplotlib

try:
    import sklearn
except:
    %pip install sklearn

try:
    import hazm
except:
    %pip install hazm

try:
    import accelerate
except:
    %pip install accelerate -U


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## پیکربندی‌های اولیه نوت‌بوک و کتاب‌خانه‌ها


In [2]:
# Some configuration for pandas for a better display of tables
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("max_colwidth", None)

from IPython.display import HTML

# Set the font family and fallback fonts for pandas output globally

def set_pandas_font(fonts):
    css = f"""
    <style>
        table.dataframe td, table.dataframe th {{
            font-family: {fonts};
        }}
    </style>
    """
    return HTML(css)

set_pandas_font("'vazirmatn', 'Vazir', 'B Nazanin', 'Arial'")

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **بارگذاری و پیش‌پردازش‌های دیتاست‌ها**
در این قسمت، دیتاست‌ها که در قالب چهار فایل با فرمت jsonline بودند، به عنوان دیتافریم‌های pandas بارگذاری شده، و توسط کتاب‌خانه هضم، محتوای فیلد «نقد» رکوردها نرمالایز می‌شود.

In [3]:
import json
from hazm import Normalizer
from sklearn.model_selection import train_test_split

normalizer = Normalizer()

def preprocess_text(text):
    return normalizer.normalize(text)

movie_data_df = pd.read_json('./data/movie.jsonl', lines=True)
movie_train_df = pd.read_json('./data/movie_train.jsonl', lines=True)
movie_test_df = pd.read_json('./data/movie_test.jsonl', lines=True)
movie_dev_df = pd.read_json('./data/movie_dev.jsonl', lines=True)

movie_data_df['review'] = movie_data_df['review'].apply(preprocess_text)
movie_train_df['review'] = movie_train_df['review'].apply(preprocess_text)
movie_test_df['review'] = movie_test_df['review'].apply(preprocess_text)
movie_dev_df['review'] = movie_dev_df['review'].apply(preprocess_text)

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **تعریف و ترین‌کردن مدل**
در این قسمت، از ترنسفورمر ازپیش‌ترین‌شده ParsBERT، که بر مبنای BERT است، برای یادگیری احساسات روی دیتاست نظرات فیلم‌ها استفاده می‌کنیم.

In [4]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Tokenize the text
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['review'], padding='max_length', truncation=True, max_length=128)

# Define the model
model = BertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=7)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy='epoch',
    logging_dir='./logs',
    fp16=False
)

# Load and tokenize the datasets
movie_train = Dataset.from_pandas(movie_train_df)
movie_train = movie_train.map(tokenize_function, batched=True)

movie_dev = Dataset.from_pandas(movie_dev_df)
movie_dev = movie_dev.map(tokenize_function, batched=True)

data_collator=lambda data: {'input_ids': torch.stack([torch.tensor(x['input_ids']) for x in data]),
                              'attention_mask': torch.stack([torch.tensor(x['attention_mask']) for x in data]),
                              'labels': torch.tensor([int(x['label']) + 3 for x in data])}
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=movie_train,
    eval_dataset=movie_dev,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertForSequenceClassification: ['distilbert.transformer.layer.1.attention.out_lin.bias', 'vocab_layer_norm.weight', 'distilbert.transformer.layer.3.attention.v_lin.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.output_layer_norm.weight', 'distilbert.transformer.layer.1.attention.k_lin.weight', 'distilbert.transformer.layer.2.output_layer_norm.bias', 'distilbert.t

Map:   0%|          | 0/2872 [00:00<?, ? examples/s]

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.071800,0.960085


TrainOutput(global_step=718, training_loss=1.0655548605746215, metrics={'train_runtime': 97.1123, 'train_samples_per_second': 29.574, 'train_steps_per_second': 7.394, 'total_flos': 188922218649600.0, 'train_loss': 1.0655548605746215, 'epoch': 1.0})

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **ارزیابی مدل**
پس از ترین‌کردن مدل، با استفاده از تابع evaluate ترینر تعریف‌شده، عملکرد مدل را بر روی داده‌های تست بررسی می‌کنیم. نتیجه عملکرد، در خروجی گزارش شده‌است.

In [8]:
# Evaluate the model
movie_test = Dataset.from_pandas(movie_train_df)
movie_test = movie_train.map(tokenize_function, batched=True)

eval_results = trainer.evaluate(movie_test)
print(eval_results)



Map:   0%|          | 0/2872 [00:00<?, ? examples/s]

{'eval_loss': 1.0577553510665894, 'eval_runtime': 20.5792, 'eval_samples_per_second': 139.558, 'eval_steps_per_second': 34.89, 'epoch': 1.0}


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **استخراج قسمت‌های مرتبط هر جنبه، از متن نقد**
تابع قطعه‌کد زیر، با گرفتن نام یک جنبه، قسمت‌هایی از متن که به آن جنبه مرتبط هستند را استخراج کرده و توکن‌ها را در خروجی برمی‌گرداند.

In [9]:
from hazm import word_tokenize

# Define a function to extract the relevant parts of the review for each aspect
def extract_aspect_text(review, aspect):
    tokens = word_tokenize(review)
    aspect_tokens = []
    for token in tokens:
        if token in aspect:
            aspect_tokens.append(token)
    return ' '.join(aspect_tokens)


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **طبقه‌بندی احساسات مبتنی بر جنبه**
تابع زیر، به عنوان تابع نهایی، یک رشته را به عنوان نظر کاربر، به همراه فهرستی از جنبه‌هایی که برای تحلیل مد نظرند را دریافت می‌کند. خروجی تابع، طبقه‌بندی احساس، به ازای هر یک از جنبه‌های خواسته‌شده می‌باشد.

In [14]:
# Define a function to classify the sentiment of the review for each aspect
def classify_sentiment(review, aspects):
    aspect_sentiments = {}
    for aspect in aspects:
        aspect_text = extract_aspect_text(review, aspect)
        if aspect_text:
            inputs = tokenizer(aspect_text, padding='max_length', truncation=True, return_tensors='pt', max_length=128)
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
            outputs = model(**inputs)
            logits = outputs.logits.detach().cpu().numpy() # Move tensor to CPU before converting to NumPy array
            aspect_sentiments[aspect] = np.argmax(logits)
    return aspect_sentiments


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **ارزیابی دستی خروجی مدل**
در قسمت‌های بالا، مدل با استفاده از داده تست، ارزیابی شده و متریک‌های مختلف مرتبط با ارزیابی برای آن در خروجی چاپ شد. در این قسمت، برای بیان شهودی عملکرد مدل در قالب گزارش، یک متن نمونه ، به همراه جنبه‌های مختلف مورد نظر داده‌شده و خروجی مدل، که یک دیکشنری است که برای هرجنبه، جهت احساس را مشخص می‌کند چاپ شده‌است.

In [16]:
import numpy as np
# Test the function
review = 'فیلم بسیار خوبی بود. بازیگران عالی بودند و داستان جذاب بود.'
aspects = ['بازی', 'داستان', 'صحنه', 'صدا', 'فیلمبرداری', 'موسیقی', 'کارگردانی', 'کلی']
aspect_sentiments = classify_sentiment(review, aspects)
print(aspect_sentiments)



{'داستان': 0, 'فیلمبرداری': 0, 'موسیقی': 0}
